# Installing Transformer Library

In [2]:
!pip install transformers torch

# Loading the Dataset

In [3]:
import pandas as pd

train_df = pd.read_csv('train.csv', encoding='latin1')
test_df = pd.read_csv('test.csv', encoding='latin1')

In [4]:
train_df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [5]:
unique_sentiments_count = train_df['sentiment'].nunique()
unique_sentiments_count

3

In [ ]:
test_df.head()

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


# Importing Bert Model and Tockenizer

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 for binary classification

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Creating functions and class for fine tunning the LLM Model Bert

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

class SentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe[dataframe['sentiment'] != 'neutral']  # Exclude neutral class
        self.tokenizer = tokenizer
        self.max_len = max_len

        # Use LabelEncoder to encode string labels to numerical values
        self.label_encoder = LabelEncoder()
        self.data['labels'] = self.label_encoder.fit_transform(self.data['sentiment'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx]['text'])  # Update with your actual column name
        label = torch.tensor(self.data.iloc[idx]['labels'], dtype=torch.long)  # Convert label to torch tensor

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')

        return {'input_ids': encoding['input_ids'].squeeze(),
                'attention_mask': encoding['attention_mask'].squeeze(),
                'labels': label.squeeze()}

In [8]:
# Initialize datasets and dataloaders
train_dataset = SentimentDataset(train_df, tokenizer, max_len=128)
test_dataset = SentimentDataset(test_df, tokenizer, max_len=128)

<ipython-input-7-26ef4f8f00fd>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['labels'] = self.label_encoder.fit_transform(self.data['sentiment'])
<ipython-input-7-26ef4f8f00fd>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['labels'] = self.label_encoder.fit_transform(self.data['sentiment'])


In [9]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [10]:
# Set up the training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*3)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# Training loop,
num_epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [17]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Debugging prints
        predicted_indices = outputs.logits.argmax(dim=1)
        print(f"Labels: {labels}")
        print(f"Predicted Indices: {predicted_indices}")


        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss}')


Epoch 1/3:   0%|          | 1/2046 [00:00<09:30,  3.59it/s]

Labels: tensor([1, 1, 1, 1, 0, 1, 1, 1], device='cuda:0')
Predicted Indices: tensor([1, 1, 1, 1, 0, 1, 1, 1], device='cuda:0')


Epoch 1/3:   0%|          | 2/2046 [00:00<07:58,  4.27it/s]

Labels: tensor([1, 1, 0, 0, 1, 0, 0, 1], device='cuda:0')
Predicted Indices: tensor([1, 1, 0, 0, 1, 0, 0, 1], device='cuda:0')
Labels: tensor([1, 0, 1, 0, 1, 1, 0, 0], device='cuda:0')
Predicted Indices: tensor([1, 0, 1, 0, 1, 1, 0, 0], device='cuda:0')


Epoch 1/3:   0%|          | 4/2046 [00:00<06:53,  4.94it/s]

...

Labels: tensor([0, 0, 1, 1, 1, 0, 1, 0], device='cuda:0')
Predicted Indices: tensor([0, 0, 1, 1, 1, 0, 1, 0], device='cuda:0')


Epoch 3/3: 100%|█████████▉| 2045/2046 [06:14<00:00,  5.46it/s]

Labels: tensor([1, 1, 0], device='cuda:0')
Predicted Indices: tensor([1, 1, 0], device='cuda:0')


Epoch 3/3: 100%|██████████| 2046/2046 [06:14<00:00,  5.46it/s]

Epoch 3/3, Average Loss: 0.030241137386797193


In [18]:
from sklearn.metrics import accuracy_score
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].numpy()

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels)

accuracy = accuracy_score(all_labels, all_preds)
accuracy

Evaluating: 100%|██████████| 424/424 [00:25<00:00, 16.36it/s]


0.589069423929099

In [14]:
accuracy

0.5896602658788774